In [7]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [8]:
# Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Classes
train_data = datasets.ImageFolder("images")
classes = train_data.classes
print(f"Classes: {classes}")

Using device: cpu
Classes: ['rose', 'sunflower', 'tulip']


In [9]:
# Transformations (data augmentation simple + normalisation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),          # Redimensionne les images à 224x224
    transforms.RandomHorizontalFlip(),      # Flip horizontal aléatoire (augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Moyenne ImageNet
                         [0.229, 0.224, 0.225])  # Écart type ImageNet
])

In [10]:
# Chargement dataset
data_dir = "./images"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Modèle : on prend un ResNet18 pré-entraîné sur ImageNet, on adapte la dernière couche
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))  # 10 classes
model = model.to(device)

# Fonction perte et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [11]:
# Fonction d'entraînement
def train_one_epoch(dataloader, model, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    print(f"Train loss: {epoch_loss:.4f}, accuracy: {epoch_acc:.4f}")

In [12]:
# Entraînement
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_one_epoch(dataloader, model, criterion, optimizer)

# Sauvegarder le modèle
torch.save(model.state_dict(), "flower_classifier.pth")
print("Modèle sauvegardé dans flower_classifier.pth")

Epoch 1/5
Train loss: 0.0857, accuracy: 0.9726
Epoch 2/5
Train loss: 0.0150, accuracy: 0.9957
Epoch 3/5
Train loss: 0.0071, accuracy: 0.9987
Epoch 4/5
Train loss: 0.0062, accuracy: 0.9980
Epoch 5/5
Train loss: 0.0068, accuracy: 0.9983
Modèle sauvegardé dans flower_classifier.pth


In [14]:
# Define dummy input with batch size 1 and correct image size 3x224x224
tensor_torch = torch.randn(1, 3, 256, 256)

# Export the model
torch.onnx.export(
    model,
    tensor_torch,
    "flower_classifier.onnx",
    input_names = ['input'],
    output_names = ['output']
)